In [1]:
import tensorflow as tf

In [3]:
x_data = [[1,2,1,1],
          [2,1,3,2],
          [3,1,3,4],
          [4,1,5,5],
          [1,7,5,5],
          [1,2,5,6],
          [1,6,6,6],
          [1,7,7,7]]
y_data = [[0,0,1],
          [0,0,1],
          [0,0,1],
          [0,1,0],
          [0,1,0],
          [0,1,0],
          [1,0,0],
          [1,0,0]]

X = tf.placeholder("float", [None,4])
Y = tf.placeholder("float", [None,3])

W = tf.Variable(tf.random_normal([4,3]), name='weight')
b = tf.Variable(tf.random_normal([3]), name='bias')

hypothesis = tf.nn.softmax(tf.matmul(X,W) + b)
cost = tf.reduce_mean(-tf.reduce_mean(Y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(2001):
        sess.run(optimizer, feed_dict={X:x_data, Y:y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}))
    a = sess.run(hypothesis, feed_dict={X:[[1,11,7,9]]})
    print(a, sess.run(tf.argmax(a,1)))

0 1.8669536
200 0.15477738
400 0.13207245
600 0.121691585
800 0.114130154
1000 0.10797278
1200 0.10269348
1400 0.0980361
1600 0.09385435
1800 0.0900551
2000 0.08657419
[[1.0473442e-01 8.9513004e-01 1.3555068e-04]] [1]


In [11]:
import numpy as np
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:,0:-1]
y_data = xy[:,[-1]]
print(x_data.shape)

(101, 16)


In [14]:
X = tf.placeholder(tf.float32, [None,16])
Y = tf.placeholder(tf.int32, [None,1])
Y_one_hot = tf.one_hot(Y, 7)
print(Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1,7])
print(Y_one_hot)

Tensor("one_hot_2:0", shape=(?, 1, 7), dtype=float32)
Tensor("Reshape:0", shape=(?, 7), dtype=float32)


In [17]:
W = tf.Variable(tf.random_normal([16,7]), name='weight')
b = tf.Variable(tf.random_normal([7]), name='bias')
logits = tf.matmul(X,W) + b
hypothesis = tf.nn.softmax(logits)

cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(2001):
        sess.run(optimizer, feed_dict={X:x_data, Y:y_data})
        if step % 200 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={X:x_data, Y:y_data})
            print(step, loss, acc)
    pred = sess.run(prediction, feed_dict={X:x_data})
    for p, y in zip(pred, y_data.flatten()):
        print("{} Prediction: {} True Y: {}".format(p==int(y), p, int(y)))

0 5.324263 0.4059406
200 0.4790439 0.8613861
400 0.29305908 0.9207921
600 0.20498171 0.9405941
800 0.15526101 0.97029704
1000 0.12380075 0.97029704
1200 0.10226987 0.97029704
1400 0.08676911 0.980198
1600 0.07518555 1.0
1800 0.06626368 1.0
2000 0.05921511 1.0
True Prediction: 0 True Y: 0
True Prediction: 0 True Y: 0
True Prediction: 3 True Y: 3
True Prediction: 0 True Y: 0
True Prediction: 0 True Y: 0
True Prediction: 0 True Y: 0
True Prediction: 0 True Y: 0
True Prediction: 3 True Y: 3
True Prediction: 3 True Y: 3
True Prediction: 0 True Y: 0
True Prediction: 0 True Y: 0
True Prediction: 1 True Y: 1
True Prediction: 3 True Y: 3
True Prediction: 6 True Y: 6
True Prediction: 6 True Y: 6
True Prediction: 6 True Y: 6
True Prediction: 1 True Y: 1
True Prediction: 0 True Y: 0
True Prediction: 3 True Y: 3
True Prediction: 0 True Y: 0
True Prediction: 1 True Y: 1
True Prediction: 1 True Y: 1
True Prediction: 0 True Y: 0
True Prediction: 1 True Y: 1
True Prediction: 5 True Y: 5
True Prediction

In [18]:
x_data = [[0,0],
          [0,1],
          [1,0],
          [1,1]]
y_data = [[0],
          [1],
          [1],
          [0]]

X = tf.placeholder(tf.float32, shape=[None, 2])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([2,1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

hypothesis = tf.sigmoid(tf.matmul(X,W) + b) #1.0 / (1 + tf.exp(-(tf.matmul(X,W) + b)))
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))

train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        cost_val, _ = sess.run([cost, train], feed_dict={X:x_data, Y:y_data})
        if step % 200 == 0:
            print(step, cost_val)
    
    h, c, a = sess.run([hypothesis, predicted, accuracy], 
                       feed_dict={X:x_data, Y:y_data})
    print("\nHypothesis: ", h,
          "\nPredicted: ", c,
          "\nAccuracy: ", a)

0 0.8472587
200 0.7467183
400 0.7179052
600 0.70866895
800 0.70446795
1000 0.70182997
1200 0.69989985
1400 0.6984163
1600 0.6972606
1800 0.696358
2000 0.6956528
2200 0.6951021
2400 0.69467217
2600 0.69433665
2800 0.6940748
3000 0.6938707
3200 0.69371146
3400 0.69358736
3600 0.69349056
3800 0.69341505
4000 0.69335616
4200 0.69331026
4400 0.69327444
4600 0.69324654
4800 0.6932248
5000 0.6932078
5200 0.6931945
5400 0.69318414
5600 0.6931761
5800 0.69316983
6000 0.6931648
6200 0.693161
6400 0.69315803
6600 0.6931557
6800 0.6931538
7000 0.6931524
7200 0.6931513
7400 0.6931503
7600 0.6931497
7800 0.69314915
8000 0.69314873
8200 0.6931484
8400 0.69314814
8600 0.6931479
8800 0.6931478
9000 0.6931477
9200 0.6931475
9400 0.6931475
9600 0.6931474
9800 0.69314736
10000 0.69314736

Hypothesis:  [[0.4997773 ]
 [0.50030106]
 [0.49962905]
 [0.5001528 ]] 
Predicted:  [[0.]
 [1.]
 [0.]
 [1.]] 
Accuracy:  0.5


In [45]:
x_data = [[0,0],
          [0,1],
          [1,0],
          [1,1]]
y_data = [[0],
          [1],
          [1],
          [0]]

X = tf.placeholder(tf.float32, shape=[None, 2])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W1 = tf.Variable(tf.random_normal([2,2]), name='weight')
b1 = tf.Variable(tf.random_normal([2]), name='bias')
layer1 = tf.sigmoid(tf.matmul(X,W1) + b1)

W2 = tf.Variable(tf.random_normal([2,1]), name='weight')
b2 = tf.Variable(tf.random_normal([1]), name='bias')
hypothesis = tf.sigmoid(tf.matmul(layer1,W2) + b2)

cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))

train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        cost_val, _ = sess.run([cost, train], feed_dict={X:x_data, Y:y_data})
        if step % 2000 == 0:
            print(step, cost_val)
    
    h, c, a = sess.run([hypothesis, predicted, accuracy], 
                       feed_dict={X:x_data, Y:y_data})
    print("\nHypothesis: ", h,
          "\nPredicted: ", c,
          "\nAccuracy: ", a)

0 0.95456564
2000 0.6866566
4000 0.68246627
6000 0.6754901
8000 0.66402596
10000 0.64561296

Hypothesis:  [[0.4076407 ]
 [0.52330613]
 [0.5354575 ]
 [0.544583  ]] 
Predicted:  [[0.]
 [1.]
 [1.]
 [1.]] 
Accuracy:  0.75


In [ ]:
x_data = [[0,0],
          [0,1],
          [1,0],
          [1,1]]
y_data = [[0],
          [1],
          [1],
          [0]]

X = tf.placeholder(tf.float32, shape=[None, 2])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W1 = tf.Variable(tf.random_normal([2,10]), name='weight')
b1 = tf.Variable(tf.random_normal([10]), name='bias')
layer1 = tf.sigmoid(tf.matmul(X,W1) + b1)

W2 = tf.Variable(tf.random_normal([10,10]), name='weight')
b2 = tf.Variable(tf.random_normal([10]), name='bias')
layer2 = tf.sigmoid(tf.matmul(layer1,W2) + b2)

W3 = tf.Variable(tf.random_normal([10,1]), name='weight')
b3 = tf.Variable(tf.random_normal([1]), name='bias')
hypothesis = tf.sigmoid(tf.matmul(layer2,W3) + b3)

cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))

train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        cost_val, _ = sess.run([cost, train], feed_dict={X:x_data, Y:y_data})
        if step % 2000 == 0:
            print(step, cost_val)
    
    h, c, a = sess.run([hypothesis, predicted, accuracy], 
                       feed_dict={X:x_data, Y:y_data})
    print("\nHypothesis: ", h,
          "\nPredicted: ", c,
          "\nAccuracy: ", a)